# Грузим данные и распаковываем

In [1]:
!mkdir data
!wget https://s3.amazonaws.com/code2vec/data/java-small_data.tar.gz
!tar -xvzf java-small_data.tar.gz
!mv java-small data

--2020-06-02 15:11:16--  https://s3.amazonaws.com/code2vec/data/java-small_data.tar.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.248.30
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.248.30|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 382731969 (365M) [application/x-tar]
Saving to: ‘java-small_data.tar.gz’

java-small_data.tar 100%[===================>] 365.00M  32.0MB/s    in 12s     

2020-06-02 15:11:29 (30.0 MB/s) - ‘java-small_data.tar.gz’ saved [382731969/382731969]

java-small/
java-small/java-small.dict.c2v
java-small/java-small.train.c2v
java-small/java-small.val.c2v
java-small/java-small.test.c2v


# Качаем всё из нашего гита

In [0]:
rm -rf DL_project_skoltech/

In [2]:
!git clone https://github.com/Vitaly-Protasov/DL_project_skoltech

Cloning into 'DL_project_skoltech'...
remote: Enumerating objects: 119, done.
remote: Counting objects: 100% (119/119), done.
remote: Compressing objects: 100% (84/84), done.
remote: Total 119 (delta 66), reused 84 (delta 34), pack-reused 0
Receiving objects: 100% (119/119), 17.10 KiB | 8.55 MiB/s, done.
Resolving deltas: 100% (66/66), done.


In [3]:
cd DL_project_skoltech/

/content/DL_project_skoltech


# У нас пока 3 модуля, импортим их


In [0]:
import create_vocab
import data_to_tensors
import model_implementation
from train import run_epoch, train

import torch
import torch.nn as nn
import torch.nn.functional as F

# Т.к. данные их, то выглядит всё так: у нас 3 основных словаря: 
* словарь всех слов в листах
* словарь всех путей(они уже хешированы и переведены в цифры)
* словарь таргетов(названий методов)


In [0]:
dict_path = '../data/java-small/java-small.dict.c2v'
word2idx, path2idx, target2idx, idx2target = create_vocab.create_vocab(dict_path)

* 1) data_iterator Возвращает по 100 батчей(размер можно поменять при вызове функции)
* 2) снизу можете видеть как идёт обращение к данным, далее в модель подаются 3 вектора(соррри за такие названия, думаю по аргументам модели поймёте что на выходе)
* 3) ну на выходе у нас code_vector и второй вектор - вероятностей распределения между всеми labels в target_vocab(словарь имен методов) 

In [0]:
path_for_train = '../data/java-small/java-small.train.c2v'
train_iterator = data_to_tensors.DatasetBuilder(path_for_train, 
                                                    word2idx, 
                                                    path2idx, 
                                                    target2idx, batch_size = 10)

path_for_val = '../data/java-small/java-small.val.c2v'
val_iterator = data_to_tensors.DatasetBuilder(path_for_val, 
                                                    word2idx, 
                                                    path2idx, 
                                                    target2idx, batch_size = 100)

In [0]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import Dataset

In [8]:
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(DEVICE)

cuda:0


In [0]:
import random 

SEED = 1234
random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [0]:
model = model_implementation.code2vec_model(values_vocab_size = len(word2idx), 
                             paths_vocab_size = len(path2idx), 
                             labels_num = len(target2idx))

In [0]:
N_EPOCHS = 500
LR = 1e-3

optimizer = torch.optim.Adam(model.parameters(), lr=LR)
criterion = nn.CrossEntropyLoss().to(DEVICE)

list_train_loss, list_val_loss, list_train_precision, list_val_precision,list_train_recall, list_val_recall, list_train_f1, list_val_f1 = train(model = model, optimizer = optimizer,
                                                                                                                                                criterion = criterion, train_loader = train_iterator,
                                                                                                                                                val_loader = val_iterator,
                                                                                                                                                epochs = N_EPOCHS, idx2target_vocab = idx2target, scheduler=None, checkpoint=True)

Epoch 1: train loss - 1.20371, validation loss - 0.1226
	 precision - 0.00018, recall - 0.00051, f1_score - 0.00026
----------------------------------------------------------------------
Epoch 2: train loss - 1.1808, validation loss - 0.12266
	 precision - 0.00018, recall - 0.00051, f1_score - 0.00026
----------------------------------------------------------------------
Epoch 3: train loss - 1.14553, validation loss - 0.12278
	 precision - 0.00022, recall - 0.00058, f1_score - 0.00032
----------------------------------------------------------------------
Epoch 4: train loss - 1.11531, validation loss - 0.12302
	 precision - 0.00014, recall - 0.00029, f1_score - 0.00019
----------------------------------------------------------------------
Epoch 5: train loss - 1.06946, validation loss - 0.12319
	 precision - 0.00013, recall - 0.00022, f1_score - 0.00016
----------------------------------------------------------------------
Epoch 6: train loss - 1.02835, validation loss - 0.12319
	 pre